In [6]:
def generatePrice(topic):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={topic}&interval=5min&apikey=1K80GWTE6MDS6ABW'
    r = requests.get(url)
    data = r.json()
    val = list(data["Time Series (5min)"].values())[0]["4. close"]
    ts = list(data["Time Series (5min)"].keys())[0]

    message_dict = {"timestamp": time.ctime(), "close": val}
    message_bytes = json.dumps(message_dict).encode('utf-8')

    return message_dict
    return data

In [7]:
print(generatePrice("IBM"))

NameError: name 'requests' is not defined

In [ ]:
%%file transaction_producer.py

import requests
import time
import json
import logging
import socket
from datetime import datetime
from confluent_kafka import Producer
import random

# def generatePrice(topic):
#     url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={topic}&interval=5min&apikey=1K80GWTE6MDS6ABW'
#     r = requests.get(url)
#     data = r.json()
#     val = random.randint(50, 80)
#     # val = list(data["Time Series (5min)"].values())[0]["4. close"]
#     # ts = list(data["Time Series (5min)"].keys())[0]

#     message_dict = {"timestamp": time.ctime(), "close": val}
    
#     return message_dict

def generate_stock_prices(start_price = 100, volatility=0.02):

    change_percent = np.random.normal(0, volatility)
    if len(PRICE_QUEUE) == 0:
        new_price = start_price * (1 + change_percent)
    else:
        new_price = PRICE_QUEUE[-1] * (1 + change_percent)
    
    message = {"timestamp": time.ctime(), "close": new_price}
    
    return message
 
KAFKA_BROKER = 'broker:9092'
TOPIC = 'IBM'
TRANSACTION_TOPIC = 'ibm_price'
LAG = 5
# PROBABILITY_OUTLIER = 0.05
 
def create_producer():
    try:
        producer = Producer({
        "bootstrap.servers":KAFKA_BROKER,
        "client.id": socket.gethostname(),
        "enable.idempotence": True,
        "batch.size": 64000,
        "linger.ms":10,
        "acks": "all",
        "retries": 5,
        "delivery.timeout.ms":1000
        })
    except Exception as e:
        logging.exception("nie mogę utworzyć producenta")
        producer = None
    return producer
 
producer = create_producer()
 
if producer is not None:
    while True:
        record = generatePrice("IBM")
        message_bytes = json.dumps(record).encode('utf-8')
        producer.produce(topic= TRANSACTION_TOPIC, value=message_bytes)
        producer.flush()
        time.sleep(LAG)

# Anomalie
Jeszcze nic nie jest zrobione z tym

In [ ]:
%%file outliers_detection.py
 
import json
import os
import time 
import numpy as np
import socket
import logging
from datetime import datetime
from joblib import load
from confluent_kafka import Producer, Consumer
from multiprocessing import Process
 
KAFKA_BROKER = 'broker:9092'
TRANSACTION_TOPIC = 'transactions'
TRANSACTOPM_CG = 'transactions'
ANOMALY_TOPIC = 'anomaly'
NUM_PARTITIONS = 3
 
MODEL_PATH = os.path.abspath('isolation_forest.joblib')
 
def create_producer():
    try:
        producer = Producer({
        "bootstrap.servers":KAFKA_BROKER,
        "client.id": socket.gethostname(),
        "enable.idempotence": True,
        "batch.size": 64000,
        "linger.ms":10,
        "acks": "all",
        "retries": 5,
        "delivery.timeout.ms":1000
        })
    except Exception as e:
        logging.exception("nie mogę utworzyć producenta")
        producer = None
    return producer
 
def create_consumer(topic, group_id):
    try:
 
        consumer = Consumer({
          "bootstrap.servers": KAFKA_BROKER,
          "group.id": group_id,
          "client.id": socket.gethostname(),
          "isolation.level":"read_committed",
          "default.topic.config":{
                    "auto.offset.reset":"latest",
                    "enable.auto.commit": False
            }
        })
        consumer.subscribe([topic])
    except Exception as e:
        logging.exception("nie mogę utworzyć konsumenta")
        consumer = None
    return consumer
    
def detekcja_anomalii():
    consumer = create_consumer(topic=TRANSACTION_TOPIC, group_id=TRANSACTOPM_CG)
    producer = create_producer()
    clf = load(MODEL_PATH)

    stack = []
    while True:
        message = consumer.poll()
        if message is None:
            continue
        if message.error():
            logging.error(f"CONSUMER error: {message.error()}")
            continue
        record = json.loads(message.value().decode('utf-8'))
        
        if (len(stack) > 5):
            stack.append(record["price"])
            stack.pop(0)
            mvAvg = sum(stack) / len(stack)

            if 
        


        data = record['data']
        prediction = clf.predict(data)
        if prediction[0] == -1 :
            score = clf.score_samples(data)
            record["score"] = np.round(score, 3).tolist()
            _id = str(record["id"])
            record = json.dumps(record).encode("utf-8")
            producer.produce(topic=ANOMALY_TOPIC, value=record)
            producer.flush()
    consumer.close()

 
for _ in range(NUM_PARTITIONS):
    p = Process(target=detekcja_anomalii)
    p.start()

## Dodaj topic anomaly i uruchom podgląd 

```bash
kafka/bin/kafka-topics.sh --create --topic anomaly --bootstrap-server broker:9092

```
```bash
cd ~
kafka/bin/kafka-console-consumer.sh --bootstrap-server broker:9092 --topic anomaly
```


In [17]:
import collections
import numpy as np
import time

# Initialize parameters
QUEUE_LENGTH = 20

SHORT_WINDOW = 5 
LONG_WINDOW = 10

RSI_WINDOW = 14

BOLLINGER_WINDOW = 20

MACD_SHORT_WINDOW = 12
MACD_LONG_WINDOW = 26
MACD_SIGNAL_WINDOW = 9

# Initialize queue
PRICE_QUEUE = collections.deque(maxlen=QUEUE_LENGTH)

# Functions to update indicators
def calculate_sma():
    if len(PRICE_QUEUE) < LONG_WINDOW:
        return "hold"

    price_list = list(PRICE_QUEUE)
    sma_long = np.mean(price_list[-LONG_WINDOW:])
    sma_short = np.mean(price_list[-SHORT_WINDOW:])

    if sma_short > sma_long:
        return 'Buy'
    elif sma_short < sma_long:
        return 'Sell'
    else:
        return 'hold'

def calculate_rsi():
    if len(PRICE_QUEUE) < RSI_WINDOW:
        return "hold"

    price_list = list(PRICE_QUEUE)
    prices = price_list[-RSI_WINDOW:]
    deltas = np.diff(prices)
    gain = np.mean([delta for delta in deltas if delta > 0])
    loss = np.mean([-delta for delta in deltas if delta < 0])

    if loss == 0:
        rsi = 100
    else:
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))

    print(rsi)

    if rsi < 30:
        return 'Buy'
    elif rsi > 70:
        return 'Sell'
    else:
        return 'hold'

def calculate_bollinger(new_price):
    if len(PRICE_QUEUE) < BOLLINGER_WINDOW:
        return "hold"

    price_list = list(PRICE_QUEUE)
    sma = np.mean(price_list[-BOLLINGER_WINDOW:])
    std = np.std(price_list[-BOLLINGER_WINDOW:])
    upper_band = sma + 2 * std
    lower_band = sma - 2 * std

    if new_price < lower_band:
        return 'Buy'
    elif new_price > upper_band:
        return 'Sell'
    else:
        return 'hold'


def calculate_ema(prices, window):
    ema = [sum(prices[:window]) / window]
    multiplier = 2 / (window + 1)
    for price in prices[window:]:
        ema.append((price - ema[-1]) * multiplier + ema[-1])
    return ema[-1]

def calculate_macd():
    if len(PRICE_QUEUE) < MACD_LONG_WINDOW:
        return "hold"

    price_list = list(PRICE_QUEUE)
    short_ema = calculate_ema(price_list, MACD_SHORT_WINDOW)
    long_ema = calculate_ema(price_list, MACD_LONG_WINDOW)
    macd = short_ema - long_ema
    macd_signal_line = calculate_ema(price_list[-MACD_SIGNAL_WINDOW:], MACD_SIGNAL_WINDOW)

    if macd > macd_signal_line:
        return 'Buy'
    elif macd < macd_signal_line:
        return 'Sell'
    else:
        return 'hold'
    
def generate_stock_prices(start_price = 100, volatility=0.02):

    change_percent = np.random.normal(0, volatility)
    if len(PRICE_QUEUE) == 0:
        new_price = start_price * (1 + change_percent)
    else:
        new_price = PRICE_QUEUE[-1] * (1 + change_percent)
    return  new_price


for i in range(20):
    real_time_price = generate_stock_prices()
    PRICE_QUEUE.append(real_time_price)
    
id = 1
for i in range(1000):
    real_time_price = generate_stock_prices()
    PRICE_QUEUE.append(real_time_price)

    record = {
        "ID": id,
        "New Price": real_time_price,
        "Signals": {"SMA" : calculate_sma(),
                    "RSI" : calculate_rsi(),
                    "BOLLINGER" : calculate_bollinger(real_time_price),
                    "MACD" : calculate_macd(),
                    }
            }
    
    id += 1
    print(record)
    time.sleep(1)



55.04834721369055
{'ID': 1, 'New Price': 93.55889699110008, 'Signals': {'SMA': 'Sell', 'RSI': 'hold', 'BOLLINGER': 'hold', 'MACD': 'hold'}}
51.62165145044209
{'ID': 2, 'New Price': 94.83044829408657, 'Signals': {'SMA': 'Buy', 'RSI': 'hold', 'BOLLINGER': 'hold', 'MACD': 'hold'}}
46.493403588898616
{'ID': 3, 'New Price': 95.57646423755884, 'Signals': {'SMA': 'Sell', 'RSI': 'hold', 'BOLLINGER': 'hold', 'MACD': 'hold'}}
48.062431946495195
{'ID': 4, 'New Price': 96.38667806453321, 'Signals': {'SMA': 'Buy', 'RSI': 'hold', 'BOLLINGER': 'hold', 'MACD': 'hold'}}
48.467683947790455
{'ID': 5, 'New Price': 99.78287395894512, 'Signals': {'SMA': 'Buy', 'RSI': 'hold', 'BOLLINGER': 'Sell', 'MACD': 'hold'}}
56.910569816908584
{'ID': 6, 'New Price': 103.0065888107862, 'Signals': {'SMA': 'Buy', 'RSI': 'hold', 'BOLLINGER': 'Sell', 'MACD': 'hold'}}
54.81193454081806
{'ID': 7, 'New Price': 102.16160865520916, 'Signals': {'SMA': 'Buy', 'RSI': 'hold', 'BOLLINGER': 'Sell', 'MACD': 'hold'}}
56.89734678661861
{'

KeyboardInterrupt: 

Trading Rules

Simple Moving Average (SMA) Crossover
Rule: Buy when the short-term moving average crosses above the long-term moving average (bullish signal). Sell when the short-term moving average crosses below the long-term moving average (bearish signal).

Relative Strength Index (RSI)
Rule: Buy when the RSI falls below 30 (indicating the stock is oversold). Sell when the RSI rises above 70 (indicating the stock is overbought).

Bollinger Bands
Rule: Buy when the price crosses below the lower Bollinger Band. Sell when the price crosses above the upper Bollinger Band.

Moving Average Convergence Divergence (MACD)
Rule: Buy when the MACD line crosses above the signal line. Sell when the MACD line crosses below the signal line.